In [1]:
from source.utils import *
from cfn_flip import flip

In [2]:
main_template = {
	"accountId": "831650818513",
	"fragment": {
		"Parameters": {
			"RepositoryName": {
				"Type": "String",
				"Default": "template-macro"
			},
			"BranchName": {
				"Type": "String",
				"Default": "master"
			},
			"BucketName": {
				"Type": "String",
				"Default": "audioposts-site"
			},
			"TemplateKey": {
				"Type": "String",
				"Default": "test/sns-topics.yaml"
			},
			"Environment": {
				"Type": "String",
				"Default": "test"
			}
		},
		"Resources": {
			"SNSTopiInlineCondition": {
				"Type": "Template::Git",
				"Properties": {
					"Mode": "Inline",
					"Provider": "Codecommit",
					"Repo": {
						"Ref": "RepositoryName"
					},
					"Branch": {
						"Ref": "BranchName"
					},
					"Path": {
						"Ref": "TemplateKey"
					},
					"Parameters": {
						"Name": "sns-topic-template-codecommit-inline-stack",
						"Environment": {
							"Ref": "Environment"
						}
					}
				}
			},
			"SNSTopiInline": {
				"Type": "Template::Git",
				"Properties": {
					"Mode": "Inline",
					"Provider": "Codecommit",
					"Repo": {
						"Ref": "RepositoryName"
					},
					"Branch": {
						"Ref": "BranchName"
					},
					"Path": {
						"Ref": "TemplateKey"
					},
					"Parameters": {
						"Name": "sns-topic-template-codecommit-inline-stack",
						"Environment": {
							"Ref": "Environment"
						}
					}
				}
			},
			"SNSTopicS3": {
				"Type": "Template::S3",
				"Properties": {
					"Mode": "Inline",
					"Provider": "S3",
					"Bucket": {
						"Ref": "BucketName"
					},
					"Key": {
						"Ref": "TemplateKey"
					},
					"Parameters": {
						"Name": "sns-topic-template-s3-inline-stack",
						"Environment": {
							"Ref": "Environment"
						}
					}
				}
			},
			"MySNSTopic": {
				"Type": "AWS::SNS::Topic",
				"Properties": {
					"TopicName": {
						"Ref": "RepositoryName"
					}
				}
			},
			"SNSTopicNested": {
				"Type": "Template::Git",
				"Properties": {
					"Mode": "Nested",
					"Provider": "Codecommit",
					"Repo": {
						"Ref": "RepositoryName"
					},
					"Branch": {
						"Ref": "BranchName"
					},
					"Path": {
						"Ref": "TemplateKey"
					},
					"Parameters": {
						"Name": "sns-topic-template-github-nested-stack",
						"Environment": {
							"Ref": "Environment"
						}
					},
					"NotificationARNs": [{
						"Fn::GetAtt": [
							"SNSTopicS3",
							"UrgentPriorityAlarm"
						]
					}],
					"Tags": [{
						"Key": "Environment",
						"Value": {
							"Ref": "Environment"
						}
					}],
					"TimeoutInMinutes": 1,
					"TemplateBucket": {
						"Ref": "BucketName"
					},
					"TemplateKey": {
						"Ref": "TemplateKey"
					}
				}
			}
		},
		"Outputs": {
			"MySNSTopicName": {
				"Description": "The DNSName of the backup load balancer",
				"Value": {
					"Fn::GetAtt": ["MySNSTopic", "Name"]
				},
				"Condition": "CreateProdResources",
				"Export": {
					"Name": {
						"Fn::Sub": "${AWS::StackName}-MySNSTopicName"
					}
				}
			},
			"MySNSTopicArn": {
				"Description": "The Instance ID",
				"Value": {
					"Ref": "EC2Instance"
				},
				"Export": {
					"Name": {
						"Fn::Join": [
							"", [
								"arn:",
								{
									"Fn::Join": [
										"", [
											"arn:",
											{
												"Ref": "Partition"
											},
											":s3:::elasticbeanstalk-*-",
											{
												"Ref": "AWS::AccountId"
											}
										]
									]
								},
								":s3:::elasticbeanstalk-*-",
								{
									"Ref": "AWS::AccountId"
								}
							]
						]
					}
				}
			}
		}
	},
	"transformId": "831650818513::Template",
	"requestId": "a54d6c4f-a952-4a3d-b88b-3e68652932c0",
	"region": "us-east-1",
	"params": {},
	"templateParameterValues": {
		"BucketName": "audioposts-site",
		"TemplateKey": "test/sns-topics.yaml",
		"Environment": "test",
		"RepositoryName": "template-macro",
		"BranchName": "master"
	}
}

In [3]:
sns_template = {
	'AWSTemplateFormatVersion': '2010-09-09',
	'Description': 'SNS Topics for alarms',
	'Parameters': {
		'SupportEmail': {
			'Type': 'String',
			'Default': 'support+18x7@xpeppers.com'
		},
		'Name': {
			'Type': 'String'
		},
		'Environment': {
			'Type': 'String'
		}
	},
      "Conditions" : {
        "CreateProdResources" : {"Fn::Equals" : [{"Ref" : "Environment"}, "prod"]}
      },
	'Resources': {
		'UrgentPriorityAlarm': {
			'Type': 'AWS::SNS::Topic',
            'Description:' : "remote repo",
			'Properties': {
				'DisplayName': {
					'Fn::Sub': '${Name}-Urgent-Priority-Alarm-${Environment}'
				},
				'Subscription': [{
					'Endpoint': {
						'Ref': 'SupportEmail'
					},
					'Protocol': 'email'
				}],
				'TopicName': {
					'Fn::Sub': '${Name}-Urgent-Priority-Alarm-${Environment}'
				}
			}
		},
		'HighPriorityAlarm': {
			'Type': 'AWS::SNS::Topic',
			'Properties': {
				'DisplayName': {
					'Fn::Sub': '${Name}-High-Priority-Alarm-${Environment}'
				},
				'Subscription': [{
					'Endpoint': {
						'Ref': 'SupportEmail'
					},
					'Protocol': 'email'
				}],
				'TopicName': {
					'Fn::Sub': '${Name}-High-Priority-Alarm-${Environment}'
				}
			}
		},
		'MediumPriorityAlarm': {
			'Type': 'AWS::SNS::Topic',
			'Properties': {
				'DisplayName': {
					'Fn::Sub': '${Name}-Medium-Priority-Alarm-${Environment}'
				},
				'Subscription': [{
					'Endpoint': {
						'Ref': 'SupportEmail'
					},
					'Protocol': 'email'
				}],
				'TopicName': {
					'Fn::Sub': '${Name}-Medium-Priority-Alarm-${Environment}'
				}
			}
		},
		'LowPriorityAlarm': {
			'Type': 'AWS::SNS::Topic',
            'Condition' : 'CreateProdResources',
			'Properties': {
				'DisplayName': {
					'Fn::Sub': '${Name}-Low-Priority-Alarm-${Environment}'
				},
				'Subscription': [{
					'Endpoint': {
						'Ref': 'SupportEmail'
					},
					'Protocol': 'email'
				}],
				'TopicName': {
					'Fn::Sub': '${Name}-Low-Priority-Alarm-${Environment}'
				}
			}
		}
	},
	'Outputs': {
		'UrgentPriorityAlarm': {
			'Description': 'Urgent priority SNS topic ARN',
			'Value': {
				'Ref': 'UrgentPriorityAlarm'
			},
			'Export': {
				'Name': {
					'Fn::Sub': '${AWS::StackName}-urgent-${Name}'
				}
			}
		},
		'HighPriorityAlarm': {
			'Description': 'High priority SNS topic ARN',
			'Value': {
				'Ref': 'HighPriorityAlarm'
			},
			'Export': {
				'Name': {
					'Fn::Sub': '${AWS::StackName}-high-${Environment}'
				}
			}
		},
		'MediumPriorityAlarm': {
			'Description': 'Medium priority SNS topic ARN',
			'Value': {
				'Ref': 'MediumPriorityAlarm'
			},
			'Export': {
				'Name': {
					'Fn::Sub': '${AWS::StackName}-medium-${Environment}'
				}
			}
		},
		'LowPriorityAlarm': {
			'Description': 'Low priority SNS topic ARN',
			'Value': {
				'Ref': 'LowPriorityAlarm'
			},
			'Export': {
				'Name': {
                  'Fn::Join': ['-', [{'Ref': 'Environment'},'low',{'Ref': 'AWS::StackName'}]]
                }
			}
		},
		'LowPriorityAlarmSelect': {
			'Description': 'Low priority SNS topic ARN',
			'Value': {
				'Ref': 'LowPriorityAlarm'
			},
			'Export': {
				'Name': {
                  'Fn::Select': [0, [{'Ref': 'Environment'},'low',{'Ref': 'AWS::StackName'}]]
                }
			}
		},
		'LowPriorityAlarmSplit': {
			'Description': 'Low priority SNS topic ARN',
			'Value': {
				'Ref': 'LowPriorityAlarm'
			},
			'Export': {
				'Name': {
                  'Fn::Select': [0, { "Fn::Split" : [ "|" , "a|b|c" ] }]
                }
			}
		},
		'LowPriorityAlarmSplit': {
			'Description': 'Low priority SNS topic ARN',
			'Value': {
				'Ref': 'LowPriorityAlarm'
			},
			'Export': {
				'Name': {
                  'Fn::Select': [0, { "Fn::Split" : [ "|" , "a|b|c" ] }]
                }
			}
		}         
	}
}

In [4]:
tp_main_template = TemplateLoader.loads(main_template["fragment"])

In [7]:
new_template = TemplateLoader.init()
new_template.parameters = tp_main_template.parameters

for resource in tp_main_template.resources:
    if tp_main_template.resources[resource].resource_type.startswith('Template::'):
        parameters = tp_main_template.resources[resource].properties['Parameters']
        
        tp_sns_template = TemplateLoader.loads(sns_template)
        tp_sns_template.set_default_parameters(parameters)
        tp_sns_template.find_relations(to_replace=True, prefix=resource)
        
        tp_sns_template.del_parameters()
        
        new_template += tp_sns_template
    else:
        new_template.add_resource(tp_main_template.resources[resource])
        
#print(tp_sns_template.to_yaml(keep_parameters=False))

In [8]:
print(new_template.to_yaml())

Conditions:
  SNSTopiInlineConditionCreateProdResources: !Equals
    - !Ref 'Environment'
    - prod
  SNSTopiInlineCreateProdResources: !Equals
    - !Ref 'Environment'
    - prod
  SNSTopicNestedCreateProdResources: !Equals
    - !Ref 'Environment'
    - prod
  SNSTopicS3CreateProdResources: !Equals
    - !Ref 'Environment'
    - prod
Description: >-
  This template is the result of the merge. [SNSTopiInlineCondition] SNS Topics for
  alarms [SNSTopiInline] SNS Topics for alarms [SNSTopicS3] SNS Topics for alarms
  [SNSTopicNested] SNS Topics for alarms
Outputs:
  SNSTopiInlineConditionHighPriorityAlarm:
    Description: '[SNSTopiInlineCondition] High priority SNS topic ARN'
    Export:
      Name: !Join
        - '-'
        - - SNSTopiInlineCondition
          - !Join
            - ''
            - - !Ref 'AWS::StackName'
              - -high-
              - !Ref 'Environment'
    Value: !Ref 'SNSTopiInlineConditionHighPriorityAlarm'
  SNSTopiInlineConditionLowPriorityAlarm:
    

In [6]:
tp_sns_template.resources['PrefixUrgentPriorityAlarm'].properties['TopicName'].data['Fn::Join'][1][2].data

{'Ref': 'Environment'}

In [7]:
tp_sns_template.parameters = []
print(tp_sns_template.to_yaml())

AWSTemplateFormatVersion: '2010-09-09'
Conditions:
  PrefixCreateProdResources: !Equals
    - !Ref 'Environment'
    - prod
Description: '[Prefix] SNS Topics for alarms'
Outputs:
  PrefixHighPriorityAlarm:
    Description: '[Prefix] High priority SNS topic ARN'
    Export:
      Name: !Join
        - ''
        - - !Ref 'PrefixAWS::StackName'
          - -high-
          - !Ref 'Environment'
    Value: !Ref 'PrefixHighPriorityAlarm'
  PrefixLowPriorityAlarm:
    Description: '[Prefix] Low priority SNS topic ARN'
    Export:
      Name: !Join
        - '-'
        - - !Ref 'Environment'
          - low
          - !Ref 'PrefixAWS::StackName'
    Value: !Ref 'PrefixLowPriorityAlarm'
  PrefixLowPriorityAlarmSelect:
    Description: '[Prefix] Low priority SNS topic ARN'
    Export:
      Name: !Select
        - 0
        - - !Ref 'Environment'
          - low
          - !Ref 'PrefixAWS::StackName'
    Value: !Ref 'PrefixLowPriorityAlarm'
  PrefixLowPriorityAlarmSplit:
    Description: '[P

In [6]:
import re, json
regex = r"(\${[^}]+})"

def _sub_to_join(template):
    template_clone = template.copy()
    _translate_value(template_clone)
    return json.loads(json.dumps(template_clone).replace('Fn::Sub','Fn::Join'))
    
def _translate_value(template):
    if type(template) == dict:
        for key in template:
            if key == 'Fn::Sub':
                split_list = [s for s in re.split(regex, template[key]) if len(s) > 0]
                for index, value in enumerate(split_list):
                    if re.match(regex, value):
                        split_list[index] = Ref(value[2:-1]).data
                template[key] = ["",split_list]
                
            _translate_value(template[key])
    
template["fragment"] = _sub_to_join(template["fragment"])
template["fragment"]

NameError: name 'template' is not defined

In [324]:
tp_template = TemplateLoader.loads(template["fragment"])

In [9]:
from troposphere import Parameter, Ref, Output, GetAtt, Sub
from troposphere import AWSObject as Resource
from troposphere import Join, Select, Export, Base64, Cidr, FindInMap, GetAtt, GetAZs, ImportValue
import troposphere

def create_resource(key, data):
    if type(data) == dict:
        [(data_key, data_value)] = data.items()
        return create_resource(data_key, data_value)
    if type(data) == list:
        for index, item in enumerate(data):
            if type(item) == str:
                continue
            if type(item) == list:
                data[index] = create_resource(None, item)
            if type(item) == dict:
                [(item_key, item_value)] = item.items()
                data[index] = create_resource(item_key, item_value)
    if key is not None:
        key = key.replace('Fn::', '')
        if type(data) == list:
            return getattr(troposphere, key)(*data)
        if type(data) == str:
            return getattr(troposphere, key)(data)
    return data

def _fix_export(template):
    name = template.data['Name']
    for Obj in [Join, Select, Export, Base64, Cidr, FindInMap, GetAZs, ImportValue]:
        if isinstance(name, Obj):
            return template    
    [(key, value)] = name.items()
    return Export(create_resource(key, value))

def _replace_ref(template, ref, prop, extraction_dict):
    if not isinstance(ref, Ref):
        return 
    else:    
        for parameter in extraction_dict['Parameter']:
            if parameter.title == ref.data['Ref']:
                if 'Default' in parameter.properties and hasattr(template, prop):
                    setattr(template, prop, parameter.properties['Default'])
                if 'Default' in parameter.properties and prop in template:
                    template[prop] = parameter.properties['Default']
                                    
extraction_dict = {'Parameter':[], 'Ref':[]}
def _find_relations(template, replace=False):

    if type(template) in [str, int] or template is None:
        return
    
    if type(template) == dict:
        for key in template:
            _find_relations(template[key], replace)
            if replace:
                _replace_ref(template, template[key], key, extraction_dict)
            
    if type(template) == list:
        for element in template:
            _find_relations(element, replace)
    
    for Obj in [Join, Select, Export, Base64, Cidr, FindInMap, GetAZs, ImportValue]:
        if isinstance(template, Obj):
            #print(type(template), Obj)
            _find_relations(template.data, replace)
    
    if isinstance(template, Resource):
        pass
        #print(type(template), 'Resource')
        #if 'Resource' in extraction_dict:   
        #    extraction_dict['Resource'] += [template]
        #else:
        #    extraction_dict['Resource'] = [template]
        
    if isinstance(template, Output):
        pass
        #print(type(template), 'Output')
        #if 'Output' in extraction_dict:   
        #    extraction_dict['Output'] += [template]
        #else:
        #    extraction_dict['Output'] = [template]
        
    if isinstance(template, Parameter):
        print(type(template), 'Parameter')
        if not replace:
            extraction_dict['Parameter'] += [template]
        
    if isinstance(template, Ref):
        #print(type(template), 'Ref')
        if not replace:
            if 'Ref' in extraction_dict:   
                extraction_dict['Ref'] += [template]
            else:
                extraction_dict['Ref'] = [template]
        
    if isinstance(template, GetAtt):
        pass
        #print(type(template), 'GetAtt')
        #if 'GetAtt' in extraction_dict:   
        #    extraction_dict['GetAtt'] += [template]
        #else:
        #    extraction_dict['GetAtt'] = [template]
        
    if hasattr(template, 'props'):
        for prop in {**template.props, **{'Conditions':(dict, False)}}:
            processed_prop = prop if hasattr(template, prop) else prop.lower()
            if hasattr(template, processed_prop):
                snippet = getattr(template, processed_prop)
                
                if isinstance(snippet, Export):
                    snippet = _fix_export(snippet)
                    setattr(template, processed_prop, snippet)
                
                if replace:
                    _replace_ref(template, snippet, processed_prop, extraction_dict)
                
                _find_relations(snippet, replace)     

_find_relations(tp_sns_template)

<class 'troposphere.Parameter'> Parameter
<class 'troposphere.Parameter'> Parameter
<class 'troposphere.Parameter'> Parameter


In [10]:
_find_relations(sns_template, replace=True)
extraction_dict

{'Parameter': [<troposphere.Parameter at 0x11162b090>,
 'Ref': [<troposphere.Ref at 0x11162b690>,
  <troposphere.Ref at 0x111630d10>]}

In [426]:
print(tp_template.to_yaml())

Outputs:
  MySNSTopicArn:
    Description: The Instance ID
    Export:
      Name: !Join
        - ''
        - - 'arn:'
          - !Join
            - ''
            - - 'arn:'
              - !Ref 'Partition'
              - :s3:::elasticbeanstalk-*-
              - !Ref 'AWS::AccountId'
          - :s3:::elasticbeanstalk-*-
          - !Ref 'AWS::AccountId'
    Value: !Ref 'EC2Instance'
  MySNSTopicName:
    Condition: CreateProdResources
    Description: The DNSName of the backup load balancer
    Export:
      Name: !Join
        - ''
        - - !Ref 'AWS::StackName'
          - -MySNSTopicName
    Value: !GetAtt 'MySNSTopic.Name'
Parameters:
  BranchName:
    Default: master
    Type: String
  BucketName:
    Default: audioposts-site
    Type: String
  Environment:
    Default: test
    Type: String
  RepositoryName:
    Default: template-macro
    Type: String
  TemplateKey:
    Default: test/sns-topics.yaml
    Type: String
Resources:
  MySNSTopic:
    Properties:
      Topic

In [388]:
extraction_dict['Parameter'][0].properties

{'Type': 'String', 'Default': 'template-macro'}

In [302]:
getattr( tp_template.outputs['MySNSTopicArn'], 'Export').data

{'Name': <troposphere.Join at 0x10795e9d0>}

In [94]:
for prop in tp_template.props:
    print(prop)

AWSTemplateFormatVersion
Transform
Description
Parameters
Mappings
Resources
Outputs
Rules


In [49]:
tp_template.parameters['RepositoryName'].props

{'Type': (str, True),
 'Default': ((str, int, float), False),
 'NoEcho': (bool, False),
 'AllowedValues': (list, False),
 'AllowedPattern': (str, False),
 'MaxLength': (<function troposphere.validators.positive_integer(x)>, False),
 'MinLength': (<function troposphere.validators.positive_integer(x)>, False),
 'MaxValue': (<function troposphere.validators.integer(x)>, False),
 'MinValue': (<function troposphere.validators.integer(x)>, False),
 'Description': (str, False),
 'ConstraintDescription': (str, False)}

In [28]:
list(prop['RepositoryName'])

TypeError: 'Parameter' object is not iterable

In [4]:
TemplateLoader(template["fragment"])

AssertionError: TemplateLoader objects must be created using TemplateLoader.loads or TemplateLoader.init

In [5]:
# print(flip(json.dumps(sns_template), clean_up=True))

In [6]:
tp_sns_template = TemplateLoader.loads(sns_template)
print(tp_sns_template.update_prefix('Prefix').to_yaml())

AWSTemplateFormatVersion: '2010-09-09'
Conditions:
  PrefixCreateProdResources: !Equals
    - !Ref 'PrefixEnvironment'
    - prod
Description: '[Prefix] SNS Topics for alarms'
Outputs:
  PrefixHighPriorityAlarm:
    Description: High priority SNS topic ARN
    Export:
      Name: !Sub 'Prefix-${AWS::StackName}-high-${PrefixEnvironment}'
    Value: !Ref 'PrefixHighPriorityAlarm'
  PrefixLowPriorityAlarm:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Join
        - '-'
        - - !Ref 'PrefixEnvironment'
          - low
          - !Ref 'AWS::StackName'
    Value: !Ref 'PrefixLowPriorityAlarm'
  PrefixLowPriorityAlarmSelect:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Select
        - 0
        - - !Ref 'PrefixEnvironment'
          - low
          - !Ref 'AWS::StackName'
    Value: !Ref 'PrefixLowPriorityAlarm'
  PrefixLowPriorityAlarmSplit:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Select
        - 0
        - 

In [12]:
from troposphere import Ref
import re
_to_replace = [("Name", "PrefixName"), ("Environment", "PrefixEnvironment")]

def _replace_value(value):
    print(value)
    for src, dst in _to_replace:
        value = re.sub('^'+src+'$', dst, value)
        value = re.sub('\${'+src+'}', '${'+dst+'}', value)
    print(value)
    return value
            
def rename_references(snippet):
    if type(snippet) in [str, int]:
        return
    
    for key in snippet:
        if type(key) == str\
            and type(snippet) == dict\
                and type(snippet[key]) == str\
                    and (key.startswith('Fn::') or key == 'Ref'):         
                        snippet[key] = _replace_value(snippet[key])
        else:
            rename_references(snippet[key] if type(snippet) == dict else key)

rename_references(sns_template)

PrefixEnvironment
PrefixEnvironment
${PrefixName}-Urgent-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Urgent-Priority-Alarm-${PrefixEnvironment}
SupportEmail
SupportEmail
${PrefixName}-Urgent-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Urgent-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-High-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-High-Priority-Alarm-${PrefixEnvironment}
SupportEmail
SupportEmail
${PrefixName}-High-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-High-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Medium-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Medium-Priority-Alarm-${PrefixEnvironment}
SupportEmail
SupportEmail
${PrefixName}-Medium-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Medium-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Low-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Low-Priority-Alarm-${PrefixEnvironment}
SupportEmail
SupportEmail
${PrefixName}-Low-Priority-Alarm-${PrefixEnvironment}
${PrefixName}-Low-Prio

In [13]:
json.dumps(sns_template)

'{"AWSTemplateFormatVersion": "2010-09-09", "Description": "SNS Topics for alarms", "Parameters": {"SupportEmail": {"Type": "String", "Default": "support+18x7@xpeppers.com"}, "Name": {"Type": "String"}, "Environment": {"Type": "String"}}, "Conditions": {"CreateProdResources": {"Fn::Equals": [{"Ref": "PrefixEnvironment"}, "prod"]}}, "Resources": {"UrgentPriorityAlarm": {"Type": "AWS::SNS::Topic", "Properties": {"DisplayName": {"Fn::Sub": "${PrefixName}-Urgent-Priority-Alarm-${PrefixEnvironment}"}, "Subscription": [{"Endpoint": {"Ref": "SupportEmail"}, "Protocol": "email"}], "TopicName": {"Fn::Sub": "${PrefixName}-Urgent-Priority-Alarm-${PrefixEnvironment}"}}}, "HighPriorityAlarm": {"Type": "AWS::SNS::Topic", "Properties": {"DisplayName": {"Fn::Sub": "${PrefixName}-High-Priority-Alarm-${PrefixEnvironment}"}, "Subscription": [{"Endpoint": {"Ref": "SupportEmail"}, "Protocol": "email"}], "TopicName": {"Fn::Sub": "${PrefixName}-High-Priority-Alarm-${PrefixEnvironment}"}}}, "MediumPriorityAla

In [14]:
tempaltes = {}
for name in tp_template.resources:
    tempaltes[name] = TemplateLoader.loads(sns_template, prefix=name)

AttributeError: 'TemplateLoader' object has no attribute '_update_prefix'

In [6]:
out = {
	"AWSTemplateFormatVersion": "2010-09-09",
	"Description": "[SNSTopicNested] SNS Topics for alarms",
	"Outputs": {
		"SNSTopicNestedHighPriorityAlarm": {
			"Description": "High priority SNS topic ARN",
			"Export": {
				"Name": {
					"Fn::Sub": "SNSTopicNested-${AWS::StackName}-high-${SNSTopicNestedEnvironment}"
				}
			},
			"Value": {
				"Ref": "SNSTopicNestedHighPriorityAlarm"
			}
		},
		"SNSTopicNestedLowPriorityAlarm": {
			"Description": "Low priority SNS topic ARN",
			"Export": {
				"Name": {
					"Fn::Sub": "SNSTopicNested-${AWS::StackName}-low-${SNSTopicNestedEnvironment}"
				}
			},
			"Value": {
				"Ref": "SNSTopicNestedLowPriorityAlarm"
			}
		},
		"SNSTopicNestedMediumPriorityAlarm": {
			"Description": "Medium priority SNS topic ARN",
			"Export": {
				"Name": {
					"Fn::Sub": "SNSTopicNested-${AWS::StackName}-medium-${SNSTopicNestedEnvironment}"
				}
			},
			"Value": {
				"Ref": "SNSTopicNestedMediumPriorityAlarm"
			}
		},
		"SNSTopicNestedUrgentPriorityAlarm": {
			"Description": "Urgent priority SNS topic ARN",
			"Export": {
				"Name": {
					"Fn::Sub": "SNSTopicNested-${AWS::StackName}-urgent-${SNSTopicNestedEnvironment}"
				}
			},
			"Value": {
				"Ref": "SNSTopicNestedUrgentPriorityAlarm"
			}
		}
	},
	"Parameters": {
		"SNSTopicNestedEnvironment": {
			"Type": "String"
		},
		"SNSTopicNestedName": {
			"Type": "String"
		},
		"SNSTopicNestedSupportEmail": {
			"Default": "support+18x7@xpeppers.com",
			"Type": "String"
		}
	},
	"Resources": {
		"SNSTopicNestedHighPriorityAlarm": {
			"Properties": {
				"DisplayName": {
					"Fn::Sub": "${SNSTopicNestedName}-High-Priority-Alarm-${SNSTopicNestedEnvironment}"
				},
				"Subscription": [{
					"Endpoint": {
						"Ref": "SNSTopicNestedSupportEmail"
					},
					"Protocol": "email"
				}],
				"TopicName": {
					"Fn::Sub": "${SNSTopicNestedName}-High-Priority-Alarm-${SNSTopicNestedEnvironment}"
				}
			},
			"Type": "AWS::SNS::Topic"
		},
		"SNSTopicNestedLowPriorityAlarm": {
			"Properties": {
				"DisplayName": {
					"Fn::Sub": "${SNSTopicNestedName}-Low-Priority-Alarm-${SNSTopicNestedEnvironment}"
				},
				"Subscription": [{
					"Endpoint": {
						"Ref": "SNSTopicNestedSupportEmail"
					},
					"Protocol": "email"
				}],
				"TopicName": {
					"Fn::Sub": "${SNSTopicNestedName}-Low-Priority-Alarm-${SNSTopicNestedEnvironment}"
				}
			},
			"Type": "AWS::SNS::Topic"
		},
		"SNSTopicNestedMediumPriorityAlarm": {
			"Properties": {
				"DisplayName": {
					"Fn::Sub": "${SNSTopicNestedName}-Medium-Priority-Alarm-${SNSTopicNestedEnvironment}"
				},
				"Subscription": [{
					"Endpoint": {
						"Ref": "SNSTopicNestedSupportEmail"
					},
					"Protocol": "email"
				}],
				"TopicName": {
					"Fn::Sub": "${SNSTopicNestedName}-Medium-Priority-Alarm-${SNSTopicNestedEnvironment}"
				}
			},
			"Type": "AWS::SNS::Topic"
		},
		"SNSTopicNestedUrgentPriorityAlarm": {
			"Properties": {
				"DisplayName": {
					"Fn::Sub": "${SNSTopicNestedName}-Urgent-Priority-Alarm-${SNSTopicNestedEnvironment}"
				},
				"Subscription": [{
					"Endpoint": {
						"Ref": "SNSTopicNestedSupportEmail"
					},
					"Protocol": "email"
				}],
				"TopicName": {
					"Fn::Sub": "${SNSTopicNestedName}-Urgent-Priority-Alarm-${SNSTopicNestedEnvironment}"
				}
			},
			"Type": "AWS::SNS::Topic"
		}
	}
}

TemplateLoader(out)

${SNSTopicNestedName}-High-Priority-Alarm-${SNSTopicNestedEnvironment}
SNSTopicNestedSupportEmail
${SNSTopicNestedName}-High-Priority-Alarm-${SNSTopicNestedEnvironment}
${SNSTopicNestedName}-Low-Priority-Alarm-${SNSTopicNestedEnvironment}
SNSTopicNestedSupportEmail
${SNSTopicNestedName}-Low-Priority-Alarm-${SNSTopicNestedEnvironment}
${SNSTopicNestedName}-Medium-Priority-Alarm-${SNSTopicNestedEnvironment}
SNSTopicNestedSupportEmail
${SNSTopicNestedName}-Medium-Priority-Alarm-${SNSTopicNestedEnvironment}
${SNSTopicNestedName}-Urgent-Priority-Alarm-${SNSTopicNestedEnvironment}
SNSTopicNestedSupportEmail
${SNSTopicNestedName}-Urgent-Priority-Alarm-${SNSTopicNestedEnvironment}
{'Name': {'Fn::Sub': 'SNSTopicNested-${AWS::StackName}-high-${SNSTopicNestedEnvironment}'}}
SNSTopicNestedHighPriorityAlarm
{'Name': {'Fn::Sub': 'SNSTopicNested-${AWS::StackName}-low-${SNSTopicNestedEnvironment}'}}
SNSTopicNestedLowPriorityAlarm
{'Name': {'Fn::Sub': 'SNSTopicNested-${AWS::StackName}-medium-${SNSTopic

metadata
conditions
mappings
outputs
parameters
resources
rules


In [18]:
tempaltes[name].outputs['SNSTopicS3HighPriorityAlarm'].properties['Export'].data['Name']

{'Fn::Sub': 'SNSTopicS3-SNSTopiInline-SNSTopicNested-${AWS::StackName}-high'}

In [19]:
from troposphere import Template

empty_template = Template()
empty_template.description = "test"

empty_template.to_json()

'{\n    "Description": "test",\n    "Resources": {}\n}'

In [20]:
new_template = TemplateLoader.init()
    
for key in tempaltes:
    new_template += tempaltes[key]

In [22]:
print(new_template.to_yaml())

Description: This template is the result of the merge. [SNSTopicNested] SNS Topics
  for alarms [SNSTopiInline] SNS Topics for alarms [SNSTopicS3] SNS Topics for alarms
Outputs:
  SNSTopiInlineHighPriorityAlarm:
    Description: High priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-high'
    Value: !Ref 'SNSTopiInlineHighPriorityAlarm'
  SNSTopiInlineLowPriorityAlarm:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-low'
    Value: !Ref 'SNSTopiInlineLowPriorityAlarm'
  SNSTopiInlineMediumPriorityAlarm:
    Description: Medium priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-medium'
    Value: !Ref 'SNSTopiInlineMediumPriorityAlarm'
  SNSTopiInlineUrgentPriorityAlarm:
    Description: Urgent priority SNS topic ARN
    Export:
      Name: !Sub 'SNSTopiInline-SNSTopicNested-${AWS::StackName}-urgent'
    Value: !Ref '

In [7]:
print(tempaltes['SNSTopicNested'].update('SNSTopicNested'))

{
    "AWSTemplateFormatVersion": "2010-09-09",
    "Description": "[SNSTopicNested] SNS Topics for alarms\n",
    "Outputs": {
        "SNSTopicNestedHighPriorityAlarm": {
            "Description": "High priority SNS topic ARN",
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-high"
                }
            },
            "Value": {
                "Ref": "SNSTopicNestedHighPriorityAlarm"
            }
        },
        "SNSTopicNestedLowPriorityAlarm": {
            "Description": "Low priority SNS topic ARN",
            "Export": {
                "Name": {
                    "Fn::Sub": "${AWS::StackName}-low"
                }
            },
            "Value": {
                "Ref": "SNSTopicNestedLowPriorityAlarm"
            }
        },
        "SNSTopicNestedMediumPriorityAlarm": {
            "Description": "Medium priority SNS topic ARN",
            "Export": {
                "Name": {
                    "Fn::

In [7]:
tempaltes['SNSTopicNested']['outputs']['SNSTopicNestedUrgentPriorityAlarm'].title
    
#attributes    
#    'properties',
# 'propnames',
# 'props',
# 'ref',
# 'resource',
# 'template',
# 'title',
# 'to_dict',
# 'validate',
# 'validate_title']

'SNSTopicNestedUrgentPriorityAlarm'

In [8]:
print(tempaltes['SNSTopicNested'].to_yaml())

AWSTemplateFormatVersion: '2010-09-09'
Description: "[SNSTopicNested] SNS Topics for alarms\n"
Outputs:
  SNSTopicNestedHighPriorityAlarm:
    Description: High priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-high'
    Value: !Ref 'HighPriorityAlarm'
  SNSTopicNestedLowPriorityAlarm:
    Description: Low priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-low'
    Value: !Ref 'LowPriorityAlarm'
  SNSTopicNestedMediumPriorityAlarm:
    Description: Medium priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-medium'
    Value: !Ref 'MediumPriorityAlarm'
  SNSTopicNestedUrgentPriorityAlarm:
    Description: Urgent priority SNS topic ARN
    Export:
      Name: !Sub '${AWS::StackName}-urgent'
    Value: !Ref 'UrgentPriorityAlarm'
Parameters:
  SNSTopicNestedAccountName:
    Default: ''
    Type: String
  SNSTopicNestedSupportEmail:
    Default: support+18x7@xpeppers.com
    Type: String
Resources:
  SNSTopicNestedHighPriorityAlarm

In [22]:
tempaltes['SNSTopicNested'].outputs['SNSTopicNestedHighPriorityAlarm'].title

'HighPriorityAlarm'

In [17]:
dir(tempaltes['SNSTopicNested'].outputs['SNSTopicNestedHighPriorityAlarm'])

['Ref',
 '_BaseAWSObject__initialized',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_from_dict',
 '_raise_type',
 '_validate_props',
 'add_to_template',
 'attributes',
 'do_validation',
 'from_dict',
 'no_validation',
 'properties',
 'propnames',
 'props',
 'ref',
 'resource',
 'template',
 'title',
 'to_dict',
 'validate',
 'validate_title']

In [133]:
new_template.parameters = {
    **tp_sns_template.parameters, 
    **tp_sns_template.parameters, 
    **tp_sns_template.parameters
}

new_template.conditions = {
    **tp_sns_template.conditions, 
    **tp_sns_template.conditions, 
    **tp_sns_template.conditions
}

new_template.description = 'test'
#    **tp_sns_template.description, 
#    **tp_sns_template.description, 
#    **tp_sns_template.description

new_template.mappings = {
    **tp_sns_template.mappings, 
    **tp_sns_template.mappings, 
    **tp_sns_template.mappings
}

new_template.metadata = {
    **tp_sns_template.metadata, 
    **tp_sns_template.metadata, 
    **tp_sns_template.metadata
}

new_template.outputs = {
    **tp_sns_template.outputs, 
    **tp_sns_template.outputs, 
    **tp_sns_template.outputs
}

new_template.props = {
    **tp_sns_template.props, 
    **tp_sns_template.props, 
    **tp_sns_template.props
}

new_template.resources = {
    **tp_sns_template.resources, 
    **tp_sns_template.resources, 
    **tp_sns_template.resources
}

new_template.rules = {
    **tp_sns_template.rules, 
    **tp_sns_template.rules, 
    **tp_sns_template.rules
}

transform = [tp_sns_template.transform, tp_sns_template.transform, tp_sns_template.transform]
new_template.transform = transform

version = [tp_sns_template.version, tp_sns_template.version, tp_sns_template.version]
new_template.version = version

In [137]:
new_template.resources

{'UrgentPriorityAlarm': <troposphere.sns.Topic at 0x10e80f0d0>,
 'HighPriorityAlarm': <troposphere.sns.Topic at 0x10e86a1d0>,
 'MediumPriorityAlarm': <troposphere.sns.Topic at 0x10e86a4d0>,
 'LowPriorityAlarm': <troposphere.sns.Topic at 0x10e86a6d0>}